In [ ]:
import numpy as np
import pandas as pd
from numpy.random import randn
from pandas import Series, DataFrame, Index

## 'Real' World Examples

Christina was nice enough to give me a dataset and some examples in SAS for me to try and duplicate in python. 

(Most fun I've had in ages...)

## Example: Searching for Sickle Cell Anemia in any discharge diagnosis across each discharge record.  

This array will pull any record from the statewide outpatient discharge file (opcalendar2017_35) where sickle cell anemia is found across any of the 30 variables for either the Principal Diagnosis (prindiag), 28 Secondary Diagnoses (Diag1-Diag28), or the E-Code (e_code).  If true, the record is exported to the temporary dataset (opsicklecell17); if false, the record is not included in the temporary dataset.  Although there are several individual ICD-10 discharge diagnoses for sickle cell anemia, they all begin with D57.XXX; therefore, the IN: statement captures the sub diagnoses without having to type each one out.
Note: the dummy dataset I provided only has 7 diagnosis fields: Prindiag, Diag1-Diag5 and e_code, to limit the size of the file (values in Diag6-Diag28 are rare anyway).


````
data opsicklecell17;
set op17.opconcalendar2017_35;
array dx [30] prindiag diag1-diag28 e_code;
	do i = 1 to 30;
		if dx(i) in : ('D57') then do;
			output opsicklecell17;
			leave;
	 	end;
	end;
drop i;
run;
````

In [ ]:
df = pd.read_excel("PythonExample.xlsx")

In [ ]:
df.info()

In [ ]:
DIAGS = ["DIAG"+str(diag+1) for diag in range(5)]

In [ ]:
DIAGS

In [ ]:
df[['PRINDIAG', 'E_CODE'] + DIAGS]

In [ ]:
mask = (df['PRINDIAG'].notnull()) & (df['PRINDIAG'].str.contains("D57"))
for col in (['E_CODE'] + DIAGS):
    mask = (mask) | (df[col].notnull()) & (df[col].str.contains("D57"))

In [ ]:
mask

In [ ]:
df[mask]

## Example: List the Sickle Cell Anemia ED encounter counts by Sex, Race/Ethnicity and Hospital in order of frequency, for Prince George’s County Hospitals

````
proc freq data= opsicklecell17 order = freq;
tables sex race*ethnicit hospid;
where hospid in : (210003, 210051, 210060, 210062) and prindiag in : ('D57'); 
run;
````

This procedure will provide the encounter counts (in order of most to least) by sex, race/ethnicity and by hospital only occurring in Prince George’s County hospitals (hospid = 210003, 210051, 210060, 210062) AND with a principal discharge diagnosis of Sickle Cell Anemia (ICD-10 code D57).


## Cheating...

I'm going to cheat and look for some other stuff since there were only two Sickle Cell Anemia cases in this data set.  

Chritina suggested I look for 

* J069 – Acute Respiratory Infection
* R0789 – Chest Pain, Other
* R51 – Headache
* I10 – Essential Hypertension
* F1120 – Opioid Dependence

In [ ]:
new_mask = df.PRINDIAG.str.contains('R0789') & df.HOSPID.isin([210003, 210051, 210060, 210062])

In [ ]:
df[new_mask].HOSPID.value_counts()

In [ ]:
df[new_mask].AGEGRP.value_counts()

In [ ]:
df.SEX.value_counts?

In [ ]:
df[new_mask].SEX.value_counts()

In [ ]:
pd.crosstab(df[new_mask].RACE,df[new_mask].ETHNICIT)

In [ ]:
pd.crosstab(df[new_mask].RACE,df[new_mask].ETHNICIT)

## Pivot Table by  Means

Christina asked me to show how to do the mean TOT_CHG by hostpital.

This is what I did on the fly:

In [ ]:
for hos in (210003, 210051, 210060, 210062):
    print(df[df.HOSPID == hos].TOT_CHG.mean())

This is how you should do it in pandas

In [ ]:
df.pivot_table(index=['HOSPID'], values=['TOT_CHG'], aggfunc='mean')